In [1]:
!git clone https://github.com/RockatanskyV8/deep-learning.git

Cloning into 'deep-learning'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 133 (delta 79), reused 106 (delta 52), pack-reused 0
Receiving objects: 100% (133/133), 332.42 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [2]:
%cd deep-learning
!git pull

/content/deep-learning
Already up to date.


# Imports e dados

In [3]:
import pandas as pd
import numpy as np

from Reader import *
from Plots  import *
from Modelos import *

from Treinamento import Treinamento

from datetime import datetime

# Implementação de Regressão

In [4]:
class Regressor():
    def __init__(self, dataset, epochs = 2000, batch_sizes = [], early_stopping_epochs = 60, retries = 5):
        self.dataset               = dataset
        self.epochs                = epochs
        self.early_stopping_epochs = early_stopping_epochs
        self.batch_sizes           = batch_sizes
        self.retries               = retries

    def regressao(self, input_features, layers, learning_rates, criterion, best_global_model = None):
        start = datetime.now()
        current_valid_loss = 0
        best_valid_loss = np.Inf
        train_losses, valid_losses = {}, {}

        for initializations in range(0, self.retries):
            for learning_rate in learning_rates:
                for batch_size in self.batch_sizes:

                    print(f'----------\nAtempt: {initializations}\nLearning rate: {learning_rate}\nBatch size: {batch_size}\n')

                    model = GeradorRede(input_features, layers)

                    if (best_global_model is not None):
                        model.load_state_dict(torch.load(best_global_model))

                    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

                    t = Treinamento(self.dataset, self.epochs, batch_size, self.early_stopping_epochs)
                    model, train_loss, valid_loss = t.train_linear(model, optimizer, criterion)

                    # store best valid loss
                    current_valid_loss = min(valid_loss)
                    if current_valid_loss < best_valid_loss:
                        torch.save(model.state_dict(), 'best_global_model')
                        best_valid_loss = current_valid_loss
                        print('New best global model found!')

                    print(f'\nValidation loss: {current_valid_loss}\n')
        
        end = datetime.now()
        print(f'\n\n\n--------------------\nTotal training time: {end - start}')




In [5]:
def run_models(dataset, criterion, layers, 
               run_epochs = 2000, run_early_stopping_epochs = 60, 
               input_features = 3, learning_rates = [], 
               run_retries = 5, run_batch_sizes = [], 
               best_global_model = None
               ):
  
  runner = Regressor(dataset, 
                     epochs = run_epochs, 
                     batch_sizes = run_batch_sizes,
                     early_stopping_epochs = run_early_stopping_epochs, 
                     retries = run_retries )
  
  runner.regressao(input_features, layers, learning_rates, criterion, best_global_model = best_global_model)



In [7]:
random_seed = 42
input_features = 3
df = pd.read_csv('df_points.txt', sep='\t', index_col=[0])

read = Reader(df)
train, valid = read.read()

X_train, X_test, y_train, y_test = train
X_valid, X_test, y_valid, y_test = valid


In [8]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F

# Modelo 1

In [ ]:
input_features = 3
output_size = 2
p = 0.5
camadas = []
camadas.append(Camada(500, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(250, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(100, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 10, nn.ReLU()))
camadas.append(Camada(output_size, nn.Softmax(dim=-1) ))

criterion = nn.CrossEntropyLoss()

## Tentativa 1

In [ ]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.0001, 0.001],
                                run_retries = 1,
                                run_batch_sizes = [15, 30])


----------
Atempt: 0
Learning rate: 0.0001
Batch size: 15



100%|██████████| 2000/2000 [26:58<00:00,  1.24it/s]


Training finished by epochs!
Total epochs run: 2000
Best model found at epoch 2000 with valid loss 62.32840257883072 and training loss 306.1731328368187
Total training time: 0:26:58.335778
New best global model found!

Validation loss: 62.32840257883072

----------
Atempt: 0
Learning rate: 0.0001
Batch size: 30



100%|██████████| 2000/2000 [15:48<00:00,  2.11it/s]


Training finished by epochs!
Total epochs run: 2000
Best model found at epoch 2000 with valid loss 34.320438623428345 and training loss 161.48700487613678
Total training time: 0:15:48.851879
New best global model found!

Validation loss: 34.320438623428345

----------
Atempt: 0
Learning rate: 0.001
Batch size: 15



 43%|████▎     | 858/2000 [11:26<15:13,  1.25it/s]


Training interrupted by early stopping!
Total epochs run: 859
Best model found at epoch 783 with valid loss 52.25267404317856 and training loss 257.287592202425
Total training time: 0:11:26.151232

Validation loss: 52.25267404317856

----------
Atempt: 0
Learning rate: 0.001
Batch size: 30



 86%|████████▌ | 1716/2000 [13:45<02:16,  2.08it/s]

Training interrupted by early stopping!
Total epochs run: 1717
Best model found at epoch 1641 with valid loss 26.04633516073227 and training loss 128.4003620147705
Total training time: 0:13:45.844944
New best global model found!

Validation loss: 26.04633516073227




--------------------
Total training time: 1:07:59.309036


In [ ]:
model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)


0.7693333333333333
0.5462915897369385


## Tentativa 2

In [ ]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.01, 0.1],
                                run_retries = 1,
                                run_batch_sizes = [45, 90],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

print(model)

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)


----------
Atempt: 0
Learning rate: 0.01
Batch size: 45



 18%|█▊        | 369/2000 [02:19<10:16,  2.64it/s]


Training interrupted by early stopping!
Total epochs run: 370
Best model found at epoch 294 with valid loss 17.55247473716736 and training loss 84.36574000120163
Total training time: 0:02:19.600690
New best global model found!

Validation loss: 17.55247473716736

----------
Atempt: 0
Learning rate: 0.01
Batch size: 90



  4%|▍         | 80/2000 [00:21<08:46,  3.65it/s]


Training interrupted by early stopping!
Total epochs run: 81
Best model found at epoch 5 with valid loss 8.78831297159195 and training loss 42.08209019899368
Total training time: 0:00:21.936883
New best global model found!

Validation loss: 8.78831297159195

----------
Atempt: 0
Learning rate: 0.1
Batch size: 45



 17%|█▋        | 331/2000 [02:06<10:39,  2.61it/s]


Training interrupted by early stopping!
Total epochs run: 332
Best model found at epoch 256 with valid loss 17.409836500883102 and training loss 83.24432674050331
Total training time: 0:02:06.794391

Validation loss: 17.409836500883102

----------
Atempt: 0
Learning rate: 0.1
Batch size: 90



  9%|▉         | 187/2000 [00:51<08:15,  3.66it/s]


Training interrupted by early stopping!
Total epochs run: 188
Best model found at epoch 112 with valid loss 8.771987050771713 and training loss 41.91084510087967
Total training time: 0:00:51.133853
New best global model found!

Validation loss: 8.771987050771713




--------------------
Total training time: 0:05:39.600621
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=500, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=500, out_features=250, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=250, out_features=100, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=100, out_features=10, bias=True)
    (10): ReLU()
    (11): Linear(in_features=10, out_features=2, bias=True)
    (12): Softmax(dim=-1)
  )
)
0.772
0.5421345829963684


## Tentativa 3

In [ ]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.0001, 0.5],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.0001
Batch size: 100



  4%|▍         | 81/2000 [00:20<08:14,  3.88it/s]


Training interrupted by early stopping!
Total epochs run: 82
Best model found at epoch 6 with valid loss 7.742163419723511 and training loss 37.619394063949585
Total training time: 0:00:20.872956
New best global model found!

Validation loss: 7.742163419723511

----------
Atempt: 0
Learning rate: 0.0001
Batch size: 120



  6%|▌         | 112/2000 [00:26<07:29,  4.20it/s]


Training interrupted by early stopping!
Total epochs run: 113
Best model found at epoch 37 with valid loss 6.7060642540454865 and training loss 31.77322742342949
Total training time: 0:00:26.700054
New best global model found!

Validation loss: 6.7060642540454865

----------
Atempt: 0
Learning rate: 0.5
Batch size: 100



 14%|█▍        | 275/2000 [01:13<07:37,  3.77it/s]


Training interrupted by early stopping!
Total epochs run: 276
Best model found at epoch 200 with valid loss 7.706204026937485 and training loss 37.07543510198593
Total training time: 0:01:13.032663

Validation loss: 7.706204026937485

----------
Atempt: 0
Learning rate: 0.5
Batch size: 120



 11%|█▏        | 228/2000 [00:56<07:21,  4.02it/s]


Training interrupted by early stopping!
Total epochs run: 229
Best model found at epoch 153 with valid loss 6.67509526014328 and training loss 31.544893324375153
Total training time: 0:00:56.795633
New best global model found!

Validation loss: 6.67509526014328




--------------------
Total training time: 0:02:57.523195
0.7693333333333333
0.5398302674293518


## Tentativa 4

In [ ]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 4000,
                                run_early_stopping_epochs = 200,
                                input_features = 3,
                                learning_rates = [0.005, 0.5],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

print(model)

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.005
Batch size: 100



  5%|▌         | 206/4000 [00:53<16:23,  3.86it/s]


Training interrupted by early stopping!
Total epochs run: 207
Best model found at epoch 6 with valid loss 7.712075710296631 and training loss 37.16488093137741
Total training time: 0:00:53.419728
New best global model found!

Validation loss: 7.712075710296631

----------
Atempt: 0
Learning rate: 0.005
Batch size: 120



  5%|▌         | 204/4000 [00:52<16:18,  3.88it/s]


Training interrupted by early stopping!
Total epochs run: 205
Best model found at epoch 4 with valid loss 6.673852831125259 and training loss 31.085947781801224
Total training time: 0:00:52.607772
New best global model found!

Validation loss: 6.673852831125259

----------
Atempt: 0
Learning rate: 0.5
Batch size: 100



 14%|█▍        | 555/4000 [02:49<17:29,  3.28it/s]


Training interrupted by early stopping!
Total epochs run: 556
Best model found at epoch 355 with valid loss 7.66717603802681 and training loss 36.7581644654274
Total training time: 0:02:49.094083

Validation loss: 7.66717603802681

----------
Atempt: 0
Learning rate: 0.5
Batch size: 120



 14%|█▍        | 562/4000 [02:35<15:49,  3.62it/s]


Training interrupted by early stopping!
Total epochs run: 563
Best model found at epoch 362 with valid loss 6.632753282785416 and training loss 30.90534347295761
Total training time: 0:02:35.150853
New best global model found!

Validation loss: 6.632753282785416




--------------------
Total training time: 0:07:10.391927
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=500, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=500, out_features=250, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=250, out_features=100, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=100, out_features=10, bias=True)
    (10): ReLU()
    (11): Linear(in_features=10, out_features=2, bias=True)
    (12): Softmax(dim=-1)
  )
)
0.778
0.5349240899085999


# Modelo 2

In [9]:
input_features = 3
output_size = 2
p = 0.5
camadas = []
camadas.append(Camada(1500, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(1000, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 500, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 250, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 100, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(  10, nn.ReLU()))
camadas.append(Camada(output_size, nn.Softmax(dim=-1) ))

criterion = nn.CrossEntropyLoss()

## Tentativa 1

In [27]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 1000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.005, 0.01],
                                run_retries = 1,
                                run_batch_sizes = [65, 30])

----------
Atempt: 0
Learning rate: 0.005
Batch size: 65



 96%|█████████▌| 955/1000 [06:59<00:19,  2.28it/s]


Training interrupted by early stopping!
Total epochs run: 956
Best model found at epoch 880 with valid loss 12.589931309223175 and training loss 59.34288400411606
Total training time: 0:06:59.190651
New best global model found!

Validation loss: 0.5245804712176323

----------
Atempt: 0
Learning rate: 0.005
Batch size: 30



 68%|██████▊   | 681/1000 [07:15<03:24,  1.56it/s]


Training interrupted by early stopping!
Total epochs run: 682
Best model found at epoch 606 with valid loss 25.853398948907852 and training loss 126.9575944840908
Total training time: 0:07:15.994117
New best global model found!

Validation loss: 0.517067978978157

----------
Atempt: 0
Learning rate: 0.01
Batch size: 65



 57%|█████▋    | 568/1000 [04:10<03:10,  2.27it/s]


Training interrupted by early stopping!
Total epochs run: 569
Best model found at epoch 493 with valid loss 12.568896502256393 and training loss 59.22185432910919
Total training time: 0:04:10.270469

Validation loss: 0.5237040209273497

----------
Atempt: 0
Learning rate: 0.01
Batch size: 30



 44%|████▍     | 441/1000 [04:38<05:52,  1.59it/s]

Training interrupted by early stopping!
Total epochs run: 442
Best model found at epoch 366 with valid loss 25.83044382929802 and training loss 127.25754222273827
Total training time: 0:04:38.122585
New best global model found!

Validation loss: 0.5166088765859604




--------------------
Total training time: 0:23:03.695298


In [28]:
model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

0.7753333333333333
0.5408754348754883


In [29]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 1000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.009, 0.5],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.009
Batch size: 100



 10%|█         | 103/1000 [00:38<05:31,  2.71it/s]


Training interrupted by early stopping!
Total epochs run: 104
Best model found at epoch 28 with valid loss 7.752797782421112 and training loss 37.82147631049156
Total training time: 0:00:38.073656
New best global model found!

Validation loss: 0.5168531854947408

----------
Atempt: 0
Learning rate: 0.009
Batch size: 120



  8%|▊         | 84/1000 [00:30<05:35,  2.73it/s]


Training interrupted by early stopping!
Total epochs run: 85
Best model found at epoch 9 with valid loss 6.7185477912425995 and training loss 32.013407707214355
Total training time: 0:00:30.756564
New best global model found!

Validation loss: 0.5168113685571231

----------
Atempt: 0
Learning rate: 0.5
Batch size: 100



 22%|██▏       | 220/1000 [01:21<04:49,  2.70it/s]


Training interrupted by early stopping!
Total epochs run: 221
Best model found at epoch 145 with valid loss 7.704874366521835 and training loss 37.35071575641632
Total training time: 0:01:21.642076
New best global model found!

Validation loss: 0.5136582911014557

----------
Atempt: 0
Learning rate: 0.5
Batch size: 120



 11%|█         | 112/1000 [00:41<05:29,  2.69it/s]


Training interrupted by early stopping!
Total epochs run: 113
Best model found at epoch 37 with valid loss 6.660477995872498 and training loss 31.301907807588577
Total training time: 0:00:41.593615
New best global model found!

Validation loss: 0.5123444612209613




--------------------
Total training time: 0:03:12.185595
0.7766666666666666
0.5361642837524414


In [30]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.009, 0.5],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.009
Batch size: 100



  4%|▍         | 76/2000 [00:28<12:08,  2.64it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 7.694870173931122 and training loss 36.80873575806618
Total training time: 0:00:28.787834
New best global model found!

Validation loss: 0.5129913449287414

----------
Atempt: 0
Learning rate: 0.009
Batch size: 120



  4%|▍         | 76/2000 [00:27<11:31,  2.78it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 6.661393404006958 and training loss 30.89363142848015
Total training time: 0:00:27.326166
New best global model found!

Validation loss: 0.5124148772313044

----------
Atempt: 0
Learning rate: 0.5
Batch size: 100



  7%|▋         | 137/2000 [00:54<12:15,  2.53it/s]


Training interrupted by early stopping!
Total epochs run: 138
Best model found at epoch 62 with valid loss 7.673706591129303 and training loss 37.076615691185
Total training time: 0:00:54.086412
New best global model found!

Validation loss: 0.5115804394086202

----------
Atempt: 0
Learning rate: 0.5
Batch size: 120



  5%|▌         | 104/2000 [00:40<12:15,  2.58it/s]


Training interrupted by early stopping!
Total epochs run: 105
Best model found at epoch 29 with valid loss 6.646476715803146 and training loss 31.01417937874794
Total training time: 0:00:40.361084
New best global model found!

Validation loss: 0.5112674396771651




--------------------
Total training time: 0:02:30.682270
0.776
0.5378246307373047


## Tentativa 2

In [31]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.8, 0.6],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.8
Batch size: 100



  6%|▋         | 128/2000 [00:51<12:40,  2.46it/s]


Training interrupted by early stopping!
Total epochs run: 129
Best model found at epoch 53 with valid loss 7.686593562364578 and training loss 36.79275530576706
Total training time: 0:00:52.012573
New best global model found!

Validation loss: 0.5124395708243052

----------
Atempt: 0
Learning rate: 0.8
Batch size: 120



  7%|▋         | 131/2000 [00:53<12:38,  2.47it/s]


Training interrupted by early stopping!
Total epochs run: 132
Best model found at epoch 56 with valid loss 6.653810560703278 and training loss 31.046063542366028
Total training time: 0:00:53.151017
New best global model found!

Validation loss: 0.5118315815925598

----------
Atempt: 0
Learning rate: 0.6
Batch size: 100



 10%|▉         | 197/2000 [01:27<13:19,  2.25it/s]


Training interrupted by early stopping!
Total epochs run: 198
Best model found at epoch 122 with valid loss 7.662471950054169 and training loss 36.83626326918602
Total training time: 0:01:27.377406
New best global model found!

Validation loss: 0.5108314633369446

----------
Atempt: 0
Learning rate: 0.6
Batch size: 120



  4%|▍         | 90/2000 [00:40<14:21,  2.22it/s]


Training interrupted by early stopping!
Total epochs run: 91
Best model found at epoch 15 with valid loss 6.642210274934769 and training loss 30.67943823337555
Total training time: 0:00:40.622049

Validation loss: 0.5109392519180591




--------------------
Total training time: 0:03:53.279785
0.778
0.5355929136276245


## Tentativa 3

In [32]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.4, 0.6],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.4
Batch size: 100



  6%|▋         | 125/2000 [01:00<15:08,  2.06it/s]


Training interrupted by early stopping!
Total epochs run: 126
Best model found at epoch 50 with valid loss 7.6550478637218475 and training loss 36.22650092840195
Total training time: 0:01:00.601212
New best global model found!

Validation loss: 0.5103365242481231

----------
Atempt: 0
Learning rate: 0.4
Batch size: 120



  6%|▌         | 112/2000 [00:51<14:23,  2.19it/s]


Training interrupted by early stopping!
Total epochs run: 113
Best model found at epoch 37 with valid loss 6.636706382036209 and training loss 30.744967967271805
Total training time: 0:00:51.225078

Validation loss: 0.5105158755412469

----------
Atempt: 0
Learning rate: 0.6
Batch size: 100



  7%|▋         | 132/2000 [01:05<15:20,  2.03it/s]


Training interrupted by early stopping!
Total epochs run: 133
Best model found at epoch 57 with valid loss 7.676155060529709 and training loss 36.70110493898392
Total training time: 0:01:05.097316

Validation loss: 0.5117436707019806

----------
Atempt: 0
Learning rate: 0.6
Batch size: 120



  9%|▉         | 187/2000 [01:23<13:32,  2.23it/s]


Training interrupted by early stopping!
Total epochs run: 188
Best model found at epoch 112 with valid loss 6.635327070951462 and training loss 30.939048439264297
Total training time: 0:01:23.790697

Validation loss: 0.510409774688574




--------------------
Total training time: 0:04:20.824790
0.7766666666666666
0.5348919034004211


## Tentativa 4

In [33]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 75,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



  4%|▍         | 76/2000 [00:39<16:30,  1.94it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 7.658006846904755 and training loss 36.04172646999359
Total training time: 0:00:39.157664
New best global model found!

Validation loss: 0.5105337897936503

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



  4%|▍         | 76/2000 [00:36<15:14,  2.10it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 6.6311438381671906 and training loss 30.567676454782486
Total training time: 0:00:36.149623
New best global model found!

Validation loss: 0.5100879875513223

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



  4%|▍         | 76/2000 [00:39<16:41,  1.92it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 7.6582739651203156 and training loss 35.62552583217621
Total training time: 0:00:39.556577

Validation loss: 0.5105515976746877

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



  4%|▍         | 76/2000 [00:35<15:09,  2.11it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 6.632268667221069 and training loss 30.276184529066086
Total training time: 0:00:35.949300

Validation loss: 0.5101745128631592




--------------------
Total training time: 0:02:30.931873
0.7786666666666666
0.533860445022583


## Tentativa 5

In [34]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 85,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



  4%|▍         | 86/2000 [00:44<16:25,  1.94it/s]


Training interrupted by early stopping!
Total epochs run: 87
Best model found at epoch 1 with valid loss 7.65825018286705 and training loss 35.6278575360775
Total training time: 0:00:44.286508
New best global model found!

Validation loss: 0.5105500121911367

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



  4%|▍         | 86/2000 [00:40<15:11,  2.10it/s]


Training interrupted by early stopping!
Total epochs run: 87
Best model found at epoch 1 with valid loss 6.635619431734085 and training loss 30.27643394470215
Total training time: 0:00:40.982301
New best global model found!

Validation loss: 0.510432263979545

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



  4%|▍         | 86/2000 [00:43<16:13,  1.97it/s]


Training interrupted by early stopping!
Total epochs run: 87
Best model found at epoch 1 with valid loss 7.664429008960724 and training loss 35.35261684656143
Total training time: 0:00:43.743749

Validation loss: 0.5109619339307149

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



  4%|▍         | 86/2000 [00:40<15:07,  2.11it/s]


Training interrupted by early stopping!
Total epochs run: 87
Best model found at epoch 1 with valid loss 6.6356397569179535 and training loss 30.057973712682724
Total training time: 0:00:40.787323

Validation loss: 0.5104338274552271




--------------------
Total training time: 0:02:49.911499
0.7766666666666666
0.5340631008148193


## Tentativa 6

In [35]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 105,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



  5%|▌         | 106/2000 [00:54<16:10,  1.95it/s]


Training interrupted by early stopping!
Total epochs run: 107
Best model found at epoch 1 with valid loss 7.665464550256729 and training loss 35.35326638817787
Total training time: 0:00:54.352483
New best global model found!

Validation loss: 0.5110309700171153

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



  5%|▌         | 106/2000 [00:49<14:46,  2.14it/s]


Training interrupted by early stopping!
Total epochs run: 107
Best model found at epoch 1 with valid loss 6.636753648519516 and training loss 30.07506775856018
Total training time: 0:00:49.610626
New best global model found!

Validation loss: 0.5105195114245782

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



  5%|▌         | 106/2000 [00:53<16:02,  1.97it/s]


Training interrupted by early stopping!
Total epochs run: 107
Best model found at epoch 1 with valid loss 7.66883197426796 and training loss 35.145259857177734
Total training time: 0:00:53.884081

Validation loss: 0.5112554649511973

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



  5%|▌         | 106/2000 [00:50<14:59,  2.11it/s]


Training interrupted by early stopping!
Total epochs run: 107
Best model found at epoch 1 with valid loss 6.6367480754852295 and training loss 29.91011071205139
Total training time: 0:00:50.365848
New best global model found!

Validation loss: 0.5105190827296331




--------------------
Total training time: 0:03:28.341818
0.7746666666666666
0.5343038439750671


In [36]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 4000,
                                run_early_stopping_epochs = 400,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120],
                                best_global_model = 'best_global_model')

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



 13%|█▎        | 531/4000 [04:17<28:00,  2.06it/s]


Training interrupted by early stopping!
Total epochs run: 532
Best model found at epoch 131 with valid loss 7.663177728652954 and training loss 36.52207973599434
Total training time: 0:04:17.288178
New best global model found!

Validation loss: 0.5108785152435302

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



 34%|███▍      | 1369/4000 [10:24<19:59,  2.19it/s]


Training interrupted by early stopping!
Total epochs run: 1370
Best model found at epoch 969 with valid loss 6.619110524654388 and training loss 30.613845854997635
Total training time: 0:10:24.234237
New best global model found!

Validation loss: 0.5091623480503376

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



 14%|█▍        | 561/4000 [04:35<28:08,  2.04it/s]


Training interrupted by early stopping!
Total epochs run: 562
Best model found at epoch 161 with valid loss 7.641813427209854 and training loss 36.2359921336174
Total training time: 0:04:35.512066

Validation loss: 0.5094542284806569

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



 10%|█         | 401/4000 [03:12<28:43,  2.09it/s]


Training interrupted by early stopping!
Total epochs run: 402
Best model found at epoch 1 with valid loss 6.623901963233948 and training loss 30.069011926651
Total training time: 0:03:12.072274

Validation loss: 0.5095309202487652




--------------------
Total training time: 0:22:29.224216
0.778
0.5345799922943115


# Nova Tentativa

In [37]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 105,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120])

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



 16%|█▌        | 324/2000 [02:02<10:35,  2.64it/s]


Training interrupted by early stopping!
Total epochs run: 325
Best model found at epoch 219 with valid loss 7.725671678781509 and training loss 37.7504161298275
Total training time: 0:02:02.828743
New best global model found!

Validation loss: 0.515044778585434

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



 26%|██▋       | 526/2000 [03:12<08:58,  2.74it/s]


Training interrupted by early stopping!
Total epochs run: 527
Best model found at epoch 421 with valid loss 6.670173913240433 and training loss 31.571165561676025
Total training time: 0:03:12.162231
New best global model found!

Validation loss: 0.5130903010184948

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



 30%|██▉       | 597/2000 [03:45<08:49,  2.65it/s]


Training interrupted by early stopping!
Total epochs run: 598
Best model found at epoch 492 with valid loss 7.713984727859497 and training loss 37.506221413612366
Total training time: 0:03:45.320613

Validation loss: 0.5142656485239665

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



 26%|██▋       | 530/2000 [03:09<08:44,  2.80it/s]


Training interrupted by early stopping!
Total epochs run: 531
Best model found at epoch 425 with valid loss 6.6931785345077515 and training loss 31.781115025281906
Total training time: 0:03:09.166527

Validation loss: 0.514859887269827




--------------------
Total training time: 0:12:09.591527
0.772
0.5381636023521423


# Regressão com validação cruzada

In [20]:
class Regressor():
    def __init__(self, dataset, epochs = 2000, batch_sizes = [], early_stopping_epochs = 60, retries = 5):
        self.dataset               = dataset
        self.epochs                = epochs
        self.early_stopping_epochs = early_stopping_epochs
        self.batch_sizes           = batch_sizes
        self.retries               = retries

    def regressao(self, input_features, layers, learning_rates, criterion, best_global_model = None):
        start = datetime.now()
        current_valid_loss = 0
        best_valid_loss = np.Inf
        train_losses, valid_losses = {}, {}

        for initializations in range(0, self.retries):
            for learning_rate in learning_rates:
                for batch_size in self.batch_sizes:

                    print(f'----------\nAtempt: {initializations}\nLearning rate: {learning_rate}\nBatch size: {batch_size}\n')

                    model = GeradorRede(input_features, layers)

                    if (best_global_model is not None):
                        model.load_state_dict(torch.load(best_global_model))

                    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

                    t = Treinamento(self.dataset, self.epochs, batch_size, self.early_stopping_epochs)
                    model, train_loss, valid_loss = t.train_cross_validation_dataloader(model, optimizer, criterion)

                    # store best valid loss
                    current_valid_loss = min(valid_loss)
                    if current_valid_loss < best_valid_loss:
                        torch.save(model.state_dict(), 'best_global_model')
                        best_valid_loss = current_valid_loss
                        print('New best global model found!')

                    print(f'\nValidation loss: {current_valid_loss}\n')
        
        end = datetime.now()
        print(f'\n\n\n--------------------\nTotal training time: {end - start}')

In [21]:
def run_models(dataset, criterion, layers, 
               run_epochs = 2000, run_early_stopping_epochs = 60, 
               input_features = 3, learning_rates = [], 
               run_retries = 5, run_batch_sizes = [], 
               best_global_model = None
               ):
  
  runner = Regressor(dataset, 
                     epochs = run_epochs, 
                     batch_sizes = run_batch_sizes,
                     early_stopping_epochs = run_early_stopping_epochs, 
                     retries = run_retries )
  
  runner.regressao(input_features, layers, learning_rates, criterion, best_global_model = best_global_model)

In [22]:
input_features = 3
output_size = 2
p = 0.5
camadas = []
camadas.append(Camada(1500, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(1000, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 500, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 250, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 100, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(  10, nn.ReLU()))
camadas.append(Camada(output_size, nn.Softmax(dim=-1) ))

criterion = nn.CrossEntropyLoss()

In [23]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 105,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120])

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



 13%|█▎        | 255/2000 [12:35<1:26:13,  2.96s/it]


Training interrupted by early stopping!
Total epochs run: 256
Best model found at epoch 150 with valid loss 7.689506828784943 and training loss 37.6681669652462
Total training time: 0:12:36.004333
New best global model found!

Validation loss: 0.5126337885856629

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



 15%|█▌        | 306/2000 [14:21<1:19:28,  2.82s/it]


Training interrupted by early stopping!
Total epochs run: 307
Best model found at epoch 201 with valid loss 6.67201566696167 and training loss 31.742998123168945
Total training time: 0:14:21.421363

Validation loss: 0.5132319743816669

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



 20%|█▉        | 391/2000 [19:10<1:18:52,  2.94s/it]


Training interrupted by early stopping!
Total epochs run: 392
Best model found at epoch 286 with valid loss 7.704663693904877 and training loss 37.536694794893265
Total training time: 0:19:10.126558

Validation loss: 0.5136442462603251

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



 18%|█▊        | 367/2000 [16:45<1:14:33,  2.74s/it]


Training interrupted by early stopping!
Total epochs run: 368
Best model found at epoch 262 with valid loss 6.6737567484378815 and training loss 31.796676605939865
Total training time: 0:16:45.316135

Validation loss: 0.5133659037259909




--------------------
Total training time: 1:02:53.069297
0.7733333333333333
0.5368683934211731


In [24]:
input_features = 3
output_size = 2
p = 0.5
camadas = []
camadas.append(Camada(500, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(250, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada(100, nn.ReLU(), nn.Dropout(p) ))
camadas.append(Camada( 10, nn.ReLU()))
camadas.append(Camada(output_size, nn.Softmax(dim=-1) ))

criterion = nn.CrossEntropyLoss()

In [25]:
current_valid_loss = run_models(df, criterion, camadas,
                                run_epochs = 2000,
                                run_early_stopping_epochs = 105,
                                input_features = 3,
                                learning_rates = [0.09, 0.05],
                                run_retries = 1,
                                run_batch_sizes = [100, 120])

model = GeradorRede(input_features, camadas)
model.load_state_dict(torch.load('best_global_model'))

p = Plots()
acc = p.get_accuracy(model, 
                     torch.from_numpy(X_test), 
                     torch.from_numpy(y_test.to_numpy()))
print(acc)

lss = p.get_loss(model, 
                 torch.from_numpy(X_test), 
                 torch.from_numpy(y_test.to_numpy()), 
                 criterion)
print(lss)

----------
Atempt: 0
Learning rate: 0.09
Batch size: 100



 26%|██▌       | 510/2000 [03:11<09:18,  2.67it/s]


Training interrupted by early stopping!
Total epochs run: 511
Best model found at epoch 405 with valid loss 7.7053124606609344 and training loss 37.3466679751873
Total training time: 0:03:11.295481
New best global model found!

Validation loss: 0.5136874973773956

----------
Atempt: 0
Learning rate: 0.09
Batch size: 120



 25%|██▍       | 496/2000 [02:59<09:02,  2.77it/s]


Training interrupted by early stopping!
Total epochs run: 497
Best model found at epoch 391 with valid loss 6.680312663316727 and training loss 31.548508882522583
Total training time: 0:02:59.088333

Validation loss: 0.5138702048705175

----------
Atempt: 0
Learning rate: 0.05
Batch size: 100



 43%|████▎     | 856/2000 [05:24<07:14,  2.63it/s]


Training interrupted by early stopping!
Total epochs run: 857
Best model found at epoch 751 with valid loss 7.6755610704422 and training loss 37.01684898138046
Total training time: 0:05:24.951512
New best global model found!

Validation loss: 0.5117040713628133

----------
Atempt: 0
Learning rate: 0.05
Batch size: 120



 24%|██▍       | 480/2000 [02:54<09:11,  2.76it/s]


Training interrupted by early stopping!
Total epochs run: 481
Best model found at epoch 375 with valid loss 6.701828092336655 and training loss 31.842782467603683
Total training time: 0:02:54.079960

Validation loss: 0.5155252378720504




--------------------
Total training time: 0:14:29.542921
0.776
0.5372288227081299
